In [1]:
#playing with https://www.kaggle.com/c/demand-forecasting-kernels-only
#todo
#-one-hot tuning

In [ ]:
import numpy as np
import pandas as pd

### 1. 데이터 로딩 및 형식 파악

In [294]:
data = pd.read_csv('train_sales.csv')

In [295]:
data.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [296]:
data.shape

(913000, 4)

In [297]:
#각 컬럼의 유니크 데이터 파악
print('Unique value count :')
for i in data.columns:
    print(i, len(data[i].unique()))

Unique value count :
date 1826
store 10
item 50
sales 213


### 2. 데이터 프로세싱

In [299]:
dt = pd.to_datetime(data['date']).dt
data['year'] = dt.year
data['month'] = dt.month
data['day'] = dt.day
data['weekday'] = dt.weekday
data['week'] = dt.week

In [300]:
#data["median-store_item"] = data.groupby(["item", "store"])["sales"].transform("median")

In [301]:
data_label = data['sales']
data_input = data[['year', 'month', 'day', 'week', 'weekday', 'store', 'item']]

In [319]:
data_input.head()
#간단한 일자별 데이터 split

,year,month,day,week,weekday,store,item
0,0,1,1,1,1,1,1
1,0,1,2,1,2,1,1
2,0,1,3,1,3,1,1
3,0,1,4,1,4,1,1
4,0,1,5,1,5,1,1


In [318]:
data_input['year'] = data_input['year'] - np.min(data_input['year'])

/Users/user/space/9/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [303]:
data_input_array, data_label_array = np.array(data_input), np.array(data_label)
val_size = int(data_input_array.shape[0] * 0.1)
train_x, val_x = data_input_array[:-val_size], data_input_array[-val_size:]
train_y, val_y = data_label_array[:-val_size], data_label_array[-val_size:]

In [284]:
from keras import backend,optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical

In [332]:
def one_hot_encode(data, features):
    res = to_categorical(data, num_classes=features+1)
    return res

In [327]:
one_hot_encode(data_input['year'], len(data_input['year'].unique())).shape

(913000, 5)

In [316]:
data_enc_input = pd.DataFrame()

In [335]:
# year = one_hot_encode(data['year'], len(data['year'].unique())+1, std=2013)
# month = one_hot_encode(data['month'], len(data['month'].unique())+1)
# day = one_hot_encode(data['day'], len(data['day'].unique())+1)
# store = one_hot_encode(data['store'], len(data['store'].unique())+1)
# item = one_hot_encode(data['item'], len(data['item'].unique())+1)
data_input_array = []
for i in data_input.columns:
    data_input_array.append(one_hot_encode(data_input[i], len(data_input[i].unique())))

In [343]:
test = np.concatenate(data_input_array, axis=1)

In [346]:
test.shape

(913000, 175)

In [106]:
one_hot_encode(data['year'], len(data['year'].unique())+1, std=2013)

array([[1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.]], dtype=float32)

In [124]:
data_coded = np.concatenate([year,month,day,store,item], axis=1)

In [125]:
data_coded.shape

(913000, 113)

In [355]:
backend.clear_session()
model = Sequential()
model.add(Dense(175, input_shape=(175,)))
# for i in range(2):
#     model.add(Dense(10, activation='relu'))
#     model.add(Dropout(0.2))
model.add(Dense(1, activation='relu'))
#adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

In [356]:
model.fit(test, data_label, batch_size=512, epochs=10, verbose=1, validation_split=0.1)

Train on 821700 samples, validate on 91300 samples
Epoch 1/10
821700/821700 [==============================] - 5s 6us/step - loss: 338.6358 - mean_absolute_error: 11.7725 - val_loss: 425.6340 - val_mean_absolute_error: 16.3738
Epoch 2/10
821700/821700 [==============================] - 4s 5us/step - loss: 102.8702 - mean_absolute_error: 7.9373 - val_loss: 433.9333 - val_mean_absolute_error: 16.4489
Epoch 3/10
821700/821700 [==============================] - 4s 5us/step - loss: 102.8490 - mean_absolute_error: 7.9380 - val_loss: 431.9685 - val_mean_absolute_error: 16.4327
Epoch 4/10
821700/821700 [==============================] - 4s 5us/step - loss: 102.8385 - mean_absolute_error: 7.9385 - val_loss: 424.5389 - val_mean_absolute_error: 16.3642
Epoch 5/10
821700/821700 [==============================] - 5s 6us/step - loss: 102.8134 - mean_absolute_error: 7.9364 - val_loss: 428.0768 - val_mean_absolute_error: 16.3956
Epoch 6/10
821700/821700 [==============================] - 4s 5us/step -